<a href="https://colab.research.google.com/github/takatakamanbou/MVA/blob/2023/omake05.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# MVA2023 omake05

<img width=64 src="https://www-tlab.math.ryukoku.ac.jp/~takataka/course/MVA/MVA-logo.png"> https://www-tlab.math.ryukoku.ac.jp/wiki/?MVA/2023


----
## おまけ課題 05 Python + OpenCV 入門
----

画像処理してみましょー

ここでは，OpenCV というライブラリを利用して画像処理してみます．OpenCV は，**コンピュータビジョン**のためのライブラリです．「コンピュータビジョン」(Computer Vision)は，人間のような視覚機能をコンピュータ・ロボットに持たせるための技術です．ここでは画像処理に限定して紹介しますが，それ以外にも様々なコンピュータビジョン関係の手法をC++とPythonで使うことができます．

自分のPCの Python 環境で OpenCV を使いたい場合は，NumPy等の場合と同じで，Python とは別にインストール作業が必要です．興味のあるひとは takataka に聞いてください．Google Colab の環境には最初からインストールされています．

----
### とりあえず画像を読み込んで表示させてみる



OpenCV と NumPy のモジュールをインポートします． OpenCV は画像を NumPy の array として扱うので， ついでに NumPy のモジュールもインポートしときます．

In [ ]:
import cv2  # cv2 モジュール（OpenCVのモジュール）をインポート
import numpy as np  # numpy モジュールをインポートして np という名前で呼べるようにする

画像を入手．最初は，takataka のウェブサーバからゲットしたのを使うことにします．

In [ ]:
# takataka のウェブサーバから uni3-192.144.jpg という名前の画像を Colab 環境へ転送
! wget -nc https://www-tlab.math.ryukoku.ac.jp/~takataka/course/AProg/uni3-192x144.jpg
! ls

自分のPCにPython + NumPy + OpenCV をインストールした環境では，OpenCV の関数 `cv2.imshow` を使って画像を表示できるのですが， Colab ではエラーになります．そこで，どちらでも動く関数 `imshow` というのを定義しておくことにします．普通の環境なら `cv2.imshow` 呼ぶだけ．

In [ ]:
# OpenCVの形式の画像を表示する関数
#
def imshow(img):
    try:
        from google.colab.patches import cv2_imshow
        cv2_imshow(img) # Colab上で実行している場合
    except:
        cv2.imshow(img)  # それ以外の場合

画像を読み込んで表示させてみましょう． OpenCV の `cv2.imread` 関数は，引数に指定されたファイル名の画像を読み込んで，画素値の入った NumPy array を返します．画像のファイル形式としては，JPEG, PNG等様々なものに対応しています．

In [ ]:
# cv2.imread は指定された名前の画像ファイルを読み込む関数．画素値の入った Numpy array を返す
imgUni3 = cv2.imread('uni3-192x144.jpg')
imshow(imgUni3)

変数 `imgUni3` の中身と shape を眺めてみましょう．

In [ ]:
print(imgUni3)
print(imgUni3.shape)

この画像は幅192画素，高さ144画素のカラー画像です．一つの画素は 青(B)，緑(G)，赤(R) の3色の数値の大きさで表現されています．各色の値は $0$ から $255$ までの $256 (=2^8)$ 段階の整数値です（「余談」参照）．

例えば，上記の最後の方に出力されている `[99 102 100]` というのは，この画像のとある位置の色が，「青成分が 99，緑成分が 102，赤成分が100」でできていることを表しています．色を表す数字は小さいほど黒に近く，大きいほどその色味が強くなります．`[0 0 0]` は黒， `[0 0 255]`は赤，`[255 255 255]` は白，`[0 255 255]` は黄色．

余談: この形式の場合， $256^3 = 2^{24} = 16777216$ 通りの色を表せることになります．また，一つの画素のデータ量は 24 [bit] です．

上記の出力の最後に `imgUni3.shape` の値がありますが，見ての通り， `imgUni3` は $144\times 192\times 3$ の大きさの3次元配列です． 画素値は画像の左上から右に向かってならんでいます．したがって，

In [ ]:
print(imgUni3[50, 100, :])

は，この画像の上から50画素，左から100画素の位置のB, G, Rの値です．

----
### 画像を加工してみる


次のセルを実行するとどんな画像が表示されるでしょうか？

In [ ]:
img2 = 255 - imgUni3
imshow(img2)

化け猫になりました．

画素値は 0 から 255 の整数値ですので，上記の計算を行うと，色が反転します．赤`[0 0 255]`がシアン（青緑）`[255 255 0]`になる，といった具合です．画像は色を表す数値の集まりですから，数値を変えれば見た目が変わるわけです．

ここで扱っている画像はただの NumPy array ですので，自分で地道にプログラムを書いていろんな画像処理を行うことができます．ですが，ここでは OpenCV の機能を利用してみることにします．

In [ ]:
# 画像の上下左右反転
img3 = cv2.flip(img2, 0) # 第2引数が 0 なら上下反転， 1 なら左右反転， -1 なら...
imshow(img3)

In [ ]:
# 画像の拡大縮小
img3 = cv2.resize(img2, (300, 400)) # 第2引数は出力の (幅, 高さ)
imshow(img3)

In [ ]:
# 画像中に円を描く．他の関数と違って引数で渡された画像そのものを書き換える
cv2.circle(img2, (185, 60), 6, color=(0, 255, 255), thickness=2)
cv2.circle(img2, (160, 135), 6, color=(0, 255, 255), thickness=2)
imshow(img2)

それぞれの関数の詳しい使い方等を知りたいときは，適当にググってみましょう．例えば `cv2.circle` の引数の意味を知りたいときは，
https://note.nkmk.me/python-opencv-draw-function/
などが参考になるでしょう．

----
### 画像の保存，Colabへのアップロード，Colabからのダウンロード

`cv2.imwrite`関数で画像をファイルに保存することができます．ファイルの形式は，引数に指定したファイル名の拡張子から自動的に判断されます．

In [ ]:
# 変数 img2 が表す画像を hoge.png というファイル名で保存
cv2.imwrite('hoge.png', img2)

普通の環境では上記を実行するだけで自分のPC上に hoge.png が出来ます．しかしColabの場合は，これだと Colab 環境上に hoge.png ができるだけなので，それを手元にダウンロードする操作も行う必要があります．

以下のセルを実行すると，Colab上のファイル `hoge.png` を自分のPCにダウンロードすることができます．ブラウザ上に「ファイルのダウンロードを許可しますか？」みたいなポップアップウィンドウが出てくるかもしれませんので，許可しましょう．

In [ ]:
# Colab からファイルをダウンロード
try:
    from google.colab import files
    files.download('hoge.png')
except:
    print('このコードは Colab 以外の環境では実行できないよ．Colabで実行してるのにこのメッセージが出る場合は，ファイル hoge.png が存在していないようです．これ以前のセルを先に実行してね')

ダウンロードした画像を自分のPCで表示させてみてください．

以下のセルを実行すると，ファイルを Colab 上へアップロードすることができます．「ファイルを選択」というボタンを押して，自分のPC上の適当な画像ファイルをアップロードしてみましょう．
ファイル名に日本語や空白が含まれているとあとあと面倒なので，あらかじめ名前をつけ直しておくとよいです．

In [ ]:
# Colab へファイルをアップロード
try:
    from google.colab import files
    rv = files.upload()
except:
    print('このコードは Colab 以外の環境では実行できないよ．')

以下のセルを実行すると，Colab上のファイル一覧を表示させることができます．アップロードしたファイルが存在していることを確認しましょう．

Colab は，Linux仮想マシン上に構築されているので，Linux のコマンドを呼び出すことができます．`!` につづけて書いている `ls` は，みなさんご存知の `ls` コマンドです．情報実習室のLinux環境や自分のPCの WSL 環境でおなじみですね．

In [ ]:
! ls

アップロードできていることが確認できたら，以下に `imshow` するセルを追加して，表示させてみましょう．

----
### 練習問題


(1) takatakaが用意したもの以外の適当な画像をアップロードし，OpenCV の機能を利用して様々な画像処理を行ってみましょう．上記で説明している以外の関数も使ってみてください．

(2) (1)でアップロードした画像の適当な範囲の画素値を反転させる処理を行いましょう．ただし...
- OpenCV の機能を使うのではなく，NumPy array に対する処理として実現しましょう．
- 繰り返しは使ってはいけません． array のスライスを使って1行で書けます．

(3) この notebook でわからないこと疑問に思うこと気になることを探して， takataka に質問しましょう．